In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./movielens/movies.csv")
rating = pd.read_csv("./movielens/ratings.csv")
tmdb_imdb = pd.read_csv("./movielens/links.csv")
tmdb_only = pd.read_excel("./check.xlsx", index_col=0)

In [3]:
movie_ratings = rating.groupby('movieId')['rating'].mean().reset_index()
movie_ratings.rename(columns={'rating': 'ave rating'}, inplace=True)

In [4]:
movie_ratings.query("movieId == 4054")

,movieId,ave rating
3026,4054,2.84375


In [5]:
df = df.merge(tmdb_imdb, on='movieId', how='inner')
df = df.merge(movie_ratings, on='movieId', how='inner')

In [6]:
df['year'] = df['title'].str.extract(r'\((\d{4})\)', expand=False)
df['year'] = pd.to_numeric(df['year'], errors='coerce')  # Converte NaN se houver erros
df['year'] = df['year'].fillna(0).astype(int)  # Substitui NaN por 0 (ou outro valor que preferir)

df['title'] = df['title'].str.replace(r'\s*\(\d{4}\)', '', regex=True)

In [7]:
df

,movieId,title,genres,imdbId,tmdbId,ave rating,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,3.920930,1995
1,2,Jumanji,Adventure|Children|Fantasy,113497,8844.0,3.431818,1995
2,3,Grumpier Old Men,Comedy|Romance,113228,15602.0,3.259615,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,114885,31357.0,2.357143,1995
4,5,Father of the Bride Part II,Comedy,113041,11862.0,3.071429,1995
...,...,...,...,...,...,...,...
9719,193581,Black Butler: Book of the Atlantic,Action|Animation|Comedy|Fantasy,5476944,432131.0,4.000000,2017
9720,193583,No Game No Life: Zero,Animation|Comedy|Fantasy,5914996,445030.0,3.500000,2017
9721,193585,Flint,Drama,6397426,479308.0,3.500000,2017
9722,193587,Bungo Stray Dogs: Dead Apple,Action|Animation,8391976,483455.0,3.500000,2018


In [8]:
df = df[df['tmdbId'].isin(tmdb_only['id'])]

In [9]:
# df = df.drop(columns=['movieId', 'imdbId', 'ave rating', 'year'])
# df = df[['tmdbId', 'title', 'genres', 'ave rating', 'year']]
df['tmdbId'] = df['tmdbId'].astype(int)

C:\Users\joao.franca\AppData\Local\Temp\ipykernel_19092\290505932.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tmdbId'] = df['tmdbId'].astype(int)


In [10]:
df

,movieId,title,genres,imdbId,tmdbId,ave rating,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,114709,862,3.920930,1995
1,2,Jumanji,Adventure|Children|Fantasy,113497,8844,3.431818,1995
5,6,Heat,Action|Crime|Thriller,113277,949,3.946078,1995
9,10,GoldenEye,Action|Adventure|Thriller,113189,710,3.496212,1995
15,16,Casino,Crime|Drama,112641,524,3.926829,1995
...,...,...,...,...,...,...,...
9539,173291,Valerian and the City of a Thousand Planets,Action|Adventure|Sci-Fi,2239822,339964,2.875000,2017
9548,173925,Seven Sisters,Sci-Fi|Thriller,1536537,406990,4.250000,2017
9549,173941,Atomic Blonde,Thriller,2406566,341013,3.000000,2017
9553,174055,Dunkirk,Action|Drama|Thriller|War,5013056,374720,3.423077,2017


In [11]:
# df.to_csv("content_movie_list_2.csv", index=False)

In [12]:
df['genres'] = df['genres'].str.split('|')

# explodeeee
df_exploded = df.explode('genres', ignore_index=True)

all_genres = sorted(set(genre for sublist in df['genres'] for genre in sublist))

for genre in all_genres:
    df_exploded[genre] = (df_exploded['genres'] == genre).astype(int)

df_exploded = df_exploded.drop(columns=['genres'])

C:\Users\joao.franca\AppData\Local\Temp\ipykernel_19092\1625963162.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].str.split('|')


In [13]:
df_exploded.columns

Index(['movieId', 'title', 'imdbId', 'tmdbId', 'ave rating', 'year',
       '(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [14]:
df_exploded = df_exploded.drop(columns=['IMAX', '(no genres listed)', 'War', 'Western', 'Musical', 'Film-Noir', 'title', 'movieId', 'imdbId'])

In [15]:
df_exploded

,tmdbId,ave rating,year,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,Horror,Mystery,Romance,Sci-Fi,Thriller
0,862,3.920930,1995,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,862,3.920930,1995,0,0,1,0,0,0,0,0,0,0,0,0,0,0
2,862,3.920930,1995,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,862,3.920930,1995,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,862,3.920930,1995,0,0,0,0,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6021,374720,3.423077,2017,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6022,353491,3.500000,2017,0,0,0,0,0,0,0,0,1,0,0,0,0,0
6023,353491,3.500000,2017,0,0,0,0,0,0,0,0,0,1,0,0,0,0
6024,353491,3.500000,2017,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [16]:
df_exploded = df_exploded[['tmdbId', 'year', 'ave rating', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Horror', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller']]

In [33]:
df_exploded.to_csv("content_item_vecs_2.csv", index=False)